# Using Sentinel-2 data <img align="right" src="../img/LivingWales_logo.png" width="190" height="200">

* **Compatibility:** Notebook currently compatible with the `WDC` environment
* **Products used:** `sen2_l2a_gcp`
* **Prerequisites:** Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_Introduction_jupyter_notebooks.ipynb)
    * Inspecting available [WDC products and measurements](02_Products_and_measurements.ipynb)
    * How to [load data from WDC](03_Loading_data.ipynb)
    * How to [plot data in WDC](04_Plotting.ipynb)


## Background
To understand the world around us, it's important to combine the key steps of loading, visualising, manipulating and analysing satellite data. One key step when using optical imagery, such as Sentinel-2, is to discard the cloudy pixels before analysing the data.

## Description
This notebook demonstrates how to use and clean Sentinel-2 data returned from a query in WDC.
It will combine many of the steps that have been covered in the other beginner's notebooks, as well as custom WDC modules.

Topics covered in this notebook include:

1. Choosing a study area.
2. Choose a study period.
3. Loading data using a geopolygon
4. Plotting data
5. Cleaning Sentinel-2 data

***

## Getting started
To run this analysis, run all the cells in the notebook starting with the "Load packages" cell. For help with running notebook cells, refer back to the [Jupyter Notebooks notebook](01_Introduction_jupyter_notebooks.ipynb).

### Load packages
The cell below imports Python packages that are used for the analysis:

* `sys` provides access to helpful support functions in the `display_tools` and `wdc_datahandling` modules. 
* `datacube` provides the ability to query and load data.
* `time` provides the ability to retrieve processing time

In [ ]:
import sys
import datacube
from time import time as time

sys.path.append("../wales_utils/data_cube_utilities")
from display_tools import map_extent, draw_select, calendar, rgb, cloud_threshold_slider
from wdc_datahandling import geom_fromdrawn, cleaning_s2, cloud_coverage, export_to_GeoTiff

### Connect to the datacube
The next step is to connect to the datacube database.
The resulting `dc` datacube object can then be used to load data.
The `app` parameter is a unique name used to identify the notebook that does not have any effect on the analysis.

In [ ]:
dc = datacube.Datacube(app="05_Sentinel2")

## Step 1: Choose a study area

One way to set the study area is to manually enter its coordinates, as demonstrated in [03_Loading_data](03_Loading_data.ipynb) notebook.

In this notebook we will use an alternative method, which uses the custom `draw_select()` tool, available in the `WDC` and developed by Living Wales, and an interactive map. 

First run the next cell and then draw the boundaries of your chosen area.

> Please: in the context of this exercise, for reasonable load times, keep the chosen site to a 2km x 2km area or less.

To draw, use either the &#9726; *'draw a rectangle'* symbol or the &#11039; *'draw a polygon'* symbol in the leftside bar, draw the boundaries of your chosen area and click on the 'Extent' button.

At any time, you can click on the bin symbol and choose 'Clear All" to delete the polygon you have drawn.




In [ ]:
m, option_widget= draw_select()
m

In [ ]:
# Get the selected option (i.e., Extent or Selection) 
selected_option = option_widget.value

# Transform the drawn boundaries to a geopolygon
my_geopolygon = geom_fromdrawn(selected_option)[0]

print("The drawn shape was be used for : " + selected_option)
print("The geopolygon of interest is the following one; ")
my_geopolygon

## Step 2: Choose a study period

In this notebook we will use an interactive method, which uses the custom `calendar()` tool, available in the `WDC`, to select the study period.

> In the context of this exercise, for reasonable load times, keep this to a range of on year or less.

In [ ]:
print('Select a start date:')
start_date = calendar()

In [ ]:
print('Select an end date:')
end_date = calendar()

## Step 3: Loading data using a geopolygon

In this example, we'll load data from Sentinel-2 satellite using the `geopolygon` parameter instead of `x`/`y`.
> **Note**: The geopolygons drawn on interactive map always have WGS84/EPSG:4326 CRS. 

When working with the Open Data Cube, it's important to load only as much data as needed.
This helps keep an analysis running quickly and avoids the notebook crashing due to insufficient memory. 

For this exercise we will only need the `RGB` and `nir` measurements, as well as the cloud mask (i.e., `scl` layer). So, let's specify this using the `measurements` argument.

In [ ]:
start_load= time()

dataset = dc.load(product= 'sen2_l2a_gcp',
                  geopolygon= my_geopolygon,
                  measurements= ['blue', 'green', 'red','nir', 'scl'],
                  time= (start_date.value, end_date.value),
                  output_crs= 'epsg:27700',
                  resolution= (-10,10),
                  dask_chunks={"y" : 2048, "x" : 2048})
end_load= time()

print("Datacube ready")
print("Took " + str(round(end_load-start_load,2)) + " seconds to request " + str(len(dataset.time
    )*len(dataset.keys())) +" images from WDC for the specified extent/period.")

Following the load step, printing the `dataset` `xarray.Dataset` object will give you insight into all of the data that was loaded.
Do this by running the next cell.

There's a lot of information to unpack, which is represented by the following aspects of the data:
- `Dimensions`: the names of data dimensions, frequently `time`, `x` and `y`, and number of entries in each
- `Coordinates`: the coordinate values for each point in the data cube
- `Data variables`: the observations loaded, typically different spectral bands from a satellite
- `Attributes`: additional useful information about the data, such as the `crs` (coordinate reference system)

In [ ]:
dataset

## Step 4: Plotting data

After loading the data, it is useful to view it to understand the resolution, which observations are impacted by cloud cover, and whether there are any obvious differences between time steps.

We use the `rgb()` function to plot the data loaded in the previous step.
The `rgb()` function maps three data variables/measurements from the loaded dataset to the red, green and blue channels that are used to make a three-colour image.

`bands = [red_channel, green_channel, blue_channel]`\
This sets the measurements that you want to use to make the image.
Any measurements can be mapped to the three channels, and different combinations highlight different features. Two common combinations are:
   * true colour: 
    `bands = ["red", "green", "blue"]`
   * false colour: `bands = ["nir", "red", "green"]`
    
For more detail about customising plots, see the [Introduction to plotting](04_Plotting.ipynb) notebook.


In [ ]:
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds to minutes depending on your period of interest and length of time requested.)")


rgb(dataset, bands=["red", "green", "blue"], col="time", robust=True, col_wrap=10)

## Step 5: Cleaning Sentinel-2 data

As we can see in the above plot, optical data are affected by cloud cover.
Filtering cloudy areas is therefore a requirement. 

The Living Wales project provides Analysis Ready Data (incl. within `WDC`), which are satellite data that have been processed to a minimum set of requirements and organized into a form that allows immediate analysis with minimum additional user effort. 

In this regard, Living Wales has developed several python tools and libraries to minimize user efforts, including library to mask clouds and normalise data. 

Within `WDC`, Sentinel-2 data are provided with same format as on the EODataDown platform (i.e., with `scl` cloud mask in a separate layer and a scaling factor of 10,000). In the next cell, we clean the Sentinel-2 data (i.e., cloud masking and reflectance normalisation) using one of the custom `WDC` libraries developed by Living Wales: `cleaning_s2()`


### Cloud masking and Reflectance normalisation

In this section, we clean the Sentinel-2 data using one of the `WDC` utilities developed by Living Wales:   `cleaning_s2()`

In [ ]:
# Let's clean the Sentinel-2 dataset (i.e., cloud masking and reflectance normalisation)
dataset_clean = cleaning_s2(dataset)

In [ ]:
# Visualise clean dataset
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds to minutes depending on your period of interest.)")

dataset_clean.nir.plot(col="time", col_wrap=10);

### Dropping useless dates

By default, all data are kept by the `cleaning_s2()` function. 
However, as we can see in the above plot, cleaning data can lead to dates having only a few, or none, remaining pixels (i.e., cloud free pixels) for the region of interest. 

In the next cells, we drop the dates which contain less than a chosen coverage of cloud-free pixels.

#### Choose a maximum cloud cover 

In [ ]:
cloud_max_threshold = cloud_threshold_slider()
cloud_max_threshold

#### Dropping dates where cloud percentage greater than cloud maximum threshold

In [ ]:
# Calculating the cloud coverage (%) for each date
cloud_percentage = cloud_coverage(dataset_clean)

# Dropping dates where cloud percentage greater than cloud maximum threshold
cloud_mask = cloud_percentage<=cloud_max_threshold.value
cloud_mask = cloud_mask.compute()
dataset_2use = dataset_clean.where(cloud_mask, drop=True)

#### Visualise clean dataset after dropping useless dates

In [ ]:
if len(dataset_2use.time) == 0:
    print("No valid images found. Try increasing the maximum cloud cover, using a larger area or different dates")
else:
    print("Plotting ...")
    print("(Please wait until images appear. This may take a few seconds)")
    dataset_2use.nir.plot(col='time', col_wrap=6);

> *Hint*: To remove the timesteps which are 100% covered with clouds,  you can directly use the fallowing function: 
`dataset_2use = dataset_clean.dropna('time', how='all')`

> *Hint*: To remove the timesteps which have at least one cloudy pixel, you can directly use the fallowing function: 
`dataset_2use = dataset_clean.dropna('time', how='any')`

## Recommended next steps

For more advanced information about working with Jupyter Notebooks or JupyterLab, see the [JupyterLab documentation](https://jupyterlab.readthedocs.io/en/stable/user/notebook.html).

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. **[Introduction to jupyter Notebooks](01_Introduction_jupyter_notebooks.ipynb)**
2. **[Products and measurements](02_Products_and_measurements.ipynb)**
3. **[Loading data in WDC](03_Loading_data.ipynb)**
4. **[Plotting](04_Plotting.ipynb)**
5. **Using Sentinel-2 data (this notebook)**
6. **[Calculating band indices](06_Calculating_band_indices.ipynb)**
7. **[Generating composites](07_Generating_composites.ipynb)**
8. **[Zonal_statistics](08_Zonal_statistics.ipynb)**


Once you have worked through the beginner's guide, you can explore the "Case Studies" directory, which provides examples of applications within Wales Open Data Cube.